In [33]:
import pdb
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torchtext.vocab import GloVe
from torchtext.data import Field, BucketIterator
from torchtext.data.metrics import bleu_score
import spacy
import fasttext.util
from nltk.translate.bleu_score import sentence_bleu

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [36]:
#Reading files

with open('trainen.txt', encoding='utf8') as f:
    eng_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('trainta.txt', encoding='utf8') as f:
    tamil_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('deven.txt', encoding='utf8') as f:
    eng_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('devta.txt', encoding='utf8') as f:
    tamil_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
embedding_glove = GloVe(name='6B', dim=100)

spacy_en = spacy.load('en_core_web_sm')

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

stop_words = [',','.','?','!',')','(',':',']','[','$','#','&','%','--']
ENG = Field(tokenize = tokenize_en, init_token='sos', eos_token = 'eos', lower=True, stop_words=stop_words)
processed_eng_train = list(map(lambda x: ENG.preprocess(x), eng_train))
processed_eng_test = list(map(lambda x: ENG.preprocess(x), eng_test))

ENG.build_vocab(processed_eng_train, vectors=embedding_glove)

In [37]:
def preprocess(processed_eng):
    
    #function to return the numericalized version of the tokenized sentences
    X = []
    for tokenized_sentence in processed_eng:
        int_sequence = [2]  #first element is the SOS token 
        for token in tokenized_sentence:
            int_sequence.append(ENG.vocab.stoi[token])
        int_sequence.append(3) #last element is the EOS token
        X.append(int_sequence)
    
    return X

# X_train and X_test are lists of lists with the integer sequences for a given sentence
X_train = preprocess(processed_eng_train)
X_test = preprocess(processed_eng_test)

In [38]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Same thing for Tamil sentences
TAM = Tokenizer()
TAM.fit_on_texts(tamil_train)
Y_train = TAM.texts_to_sequences(tamil_train)
Y_test = TAM.texts_to_sequences(tamil_test)

#adding EOS token
_ = [y.append(0) for y in Y_train]
_ = [y.append(0) for y in Y_test]

In [39]:
TAM.index_word[0] = 'EOS'
TAM.index_word[len(TAM.index_word)] = 'SOS'

In [40]:
source_vocab_size = len(ENG.vocab)
target_vocab_size = len(TAM.index_word)
print(source_vocab_size)
print(target_vocab_size)

9723
18670


In [41]:
# Code to load a model and save embeddings as a numpy array for later use
# ft = fasttext.load_model('cc.ta.300.bin')
# tamil_embedding = torch.zeros(target_vocab_size, 300)
# for i in range(target_vocab_size):
#     tamil_embedding[i] = torch.FloatTensor(ft.get_word_vector(TAM.index_word[i]))

# X = tamil_embedding.numpy()
# np.save('tam_embed.npy', X)

In [42]:
# loading pretrained weights from numpy file
tamil_embedding = np.load('tam_embed.npy')
tamil_embedding = torch.from_numpy(tamil_embedding)

In [43]:
class Encoder(nn.Module):
    
    def __init__(self, embed_size, hidden_size, ENG):
        
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding.from_pretrained(ENG.vocab.vectors)
        self.lstm = nn.LSTM(embed_size, hidden_size)
        
    def forward(self, x, enc_hidden):
        
        embedded_x = self.embed(x)  # Ts x 100
        enc_output, enc_hidden = self.lstm(embedded_x.unsqueeze(1), enc_hidden)
        return enc_output, enc_hidden

In [44]:
class Decoder(nn.Module):
    
    def __init__(self, hidden_size, n_classes, embeddings):
        
        super(Decoder, self).__init__()
        self.embed_size = embeddings.shape[1]
        self.hidden_size = hidden_size
        self.embed = nn.Embedding.from_pretrained(embeddings)
        self.lstm = nn.LSTM(embeddings.shape[1], hidden_size)
        self.fc = nn.Linear(hidden_size, n_classes)
        self.logsoftmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x, dec_hidden):
        
        dec_output, dec_hidden = self.lstm(self.embed(x).view(1,1,-1), dec_hidden)
        dec_output = self.logsoftmax(self.fc(dec_output[0]))
        
        return dec_output, dec_hidden

In [45]:
def loss_per_pair(source_sentence, target_sentence, enc_obj, dec_obj, enc_optimiser, dec_optimiser, loss_fn):
    
    Ts = source_sentence.size(-1)
    Tt = target_sentence.size(-1)
    enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))
    
    enc_optimiser.zero_grad()
    dec_optimiser.zero_grad()
    
    loss_val = 0
    
    # source_sentence is a 1-D tensor of size Ts
    enc_output, enc_hidden = enc_obj(source_sentence, enc_hidden)
    
    dec_input = torch.tensor([[target_vocab_size-1]], device=device)  #SOS token
    
    # first hidden(and cell) state of decoder is made the final hidden(and cell) state of the encoder
    dec_hidden = enc_hidden
    
    for i in range(Tt):
        
        dec_output, dec_hidden = dec_obj(dec_input, dec_hidden)
        _ , index = dec_output.topk(1)
        dec_input = index.squeeze().detach()
        #print(TAM.index_word[dec_input.item()])
            
        target_word = torch.cuda.LongTensor([target_sentence[i].item()])
        
        loss_val += loss_fn(dec_output, target_word)
        
        if dec_input.item() == 0:
            break

    loss_val.backward()
    
    nn.utils.clip_grad_norm_(enc_obj.parameters(), 0.5)
    nn.utils.clip_grad_norm_(dec_obj.parameters(), 0.5)

    enc_optimiser.step()
    dec_optimiser.step()

    return loss_val.item()/Tt

In [46]:
def train_model(sources, targets, enc_obj, dec_obj, lr=0.001, mom=0.9):
    
    loss_fn = nn.NLLLoss()
    enc_optimiser = optim.SGD(enc_obj.parameters(), lr=lr, momentum=mom)
    dec_optimiser = optim.SGD(dec_obj.parameters(), lr=lr, momentum=mom)
#     enc_optimiser = optim.Adagrad(enc_obj.parameters())
#     dec_optimiser = optim.Adagrad(dec_obj.parameters())
    
    max_epochs = 100
    old_loss = np.inf
    indices = [i for i in range(len(sources))]
    
    for epoch in range(max_epochs):
        
        # shuffling the input data manually at the start of every new epoch
        np.random.shuffle(indices)
        sources = list(np.array(sources)[indices])
        targets = list(np.array(targets)[indices])
        
        running_loss = 0.0
        num_sentences = len(sources)
        
        for i in range(num_sentences):
            source_sentence = torch.cuda.LongTensor(sources[i])
            target_sentence = torch.cuda.LongTensor(targets[i])

            loss = loss_per_pair(source_sentence, target_sentence, enc_obj, dec_obj, enc_optimiser, dec_optimiser, loss_fn)
            running_loss += loss

            if i%int(num_sentences*0.1)==0:
                print("Epoch", epoch+1, ":", (i/int(num_sentences*0.1))*10,'% done')
                print("Current loss:", running_loss)
                
        print('\nEpoch', epoch+1,"\n")
        print("Encoder obj lstm wts sum=", torch.sum(enc_obj.lstm.weight_ih_l0))
        print("Encoder obj lstm bias sum=", torch.sum(enc_obj.lstm.bias_ih_l0))
        print('Decoder obj emb wts sum=', torch.sum(dec_obj.embed.weight.data))
        print('Decoder obj lstm wts sum=', torch.sum(dec_obj.lstm.weight_ih_l0.data))
        print('Decoder obj lstm bias sum=', torch.sum(dec_obj.lstm.bias_ih_l0.data))
        print('Decoder obj linear wts sum=', torch.sum(dec_obj.fc.weight.data))
        print('Decoder obj linear bias sum=', torch.sum(dec_obj.fc.bias.data))
        
        
        if abs(running_loss-old_loss)/running_loss < 1e-3:
            print('Converged')
            break
    
        old_loss = running_loss

    print("Finished Training")

In [47]:
embedding_size = 100
hidden_size = 150
encoder = Encoder(embedding_size, hidden_size, ENG).to(device)
decoder = Decoder(hidden_size, target_vocab_size, tamil_embedding).to(device)
# encoder.load_state_dict(torch.load('q3_encoder_weights.pt'))
# decoder.load_state_dict(torch.load('q3_decoder_weights.pt'))
train_model(X_train, Y_train, encoder, decoder)

Epoch 1 : 0.0 % done
Current loss: 9.833087921142578


KeyboardInterrupt: 

In [ ]:
torch.save(encoder.state_dict(), 'q3_enc_wts.pt')
torch.save(decoder.state_dict(), 'q3_dec_wts.pt')
# Add these two lines after declaring the encoder and decoder objects and before train_model()
#train_model(X_train, Y_train, encoder, decoder)

In [23]:
def test_loss(sources, targets, enc_obj, dec_obj):
    
    loss_fn = nn.NLLLoss()
    num_sentences = len(sources)
    total_loss = 0
        
    for i in range(num_sentences):

        with torch.no_grad():

            source_sentence = torch.cuda.LongTensor(sources[i])
            target_sentence = torch.cuda.LongTensor(targets[i])
            Ts = source_sentence.size(-1)
            Tt = target_sentence.size(-1)
            enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))
            
            loss_val = 0
            
            enc_output, enc_hidden = enc_obj(source_sentence, enc_hidden)

            dec_input = torch.tensor([[target_vocab_size-1]], device=device)  #SOS token

            # first hidden state of decoder is made the final hidden state of the encoder and cell state is initialised with zeros
            dec_hidden = enc_hidden

            for i in range(Tt):

                dec_output, dec_hidden = dec_obj(dec_input, dec_hidden)
                _ , index = dec_output.topk(1)
                dec_input = index.squeeze().detach()
                if index.item() == 0:
                    break

                target_word = torch.cuda.LongTensor([target_sentence[i].item()])

                loss_val += loss_fn(dec_output, target_word)
            
            total_loss += loss_val.item()/Tt
    
    return total_loss

In [24]:
# this test loss is for 500 sentences
test_loss(X_test, Y_test, encoder, decoder)

739.9332675518084

In [25]:
def eval_bleu(enc_obj, dec_obj, source_sentence, target, target_vocab_dict):
    
    # function to return the BLEU score for a single sentence 
    
    with torch.no_grad():
        
        Ts = source_sentence.size(-1)
        enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))

        enc_output, enc_hidden = enc_obj(source_sentence, enc_hidden)

        dec_input = torch.tensor([[target_vocab_size-1]], device=device)  # SOS

        dec_hidden = enc_hidden

        predicted = []

        for i in range(60):
            dec_output, dec_hidden = dec_obj(dec_input, dec_hidden)
            _ , index = dec_output.data.topk(1)
            if index.item() == 0:
                #decoded_words.append('<EOS>')
                break
            else:
                predicted.append(target_vocab_dict[index.item()])

            dec_input = index.squeeze().detach()

    predicted = [predicted]
    target = [[target]]
    print("Predicted:",predicted)
    print("Target:",target)
    
    bleu1 = bleu_score(predicted, target, weights=(1.0, 0, 0, 0))
    bleu2 = bleu_score(predicted, target, weights=(0.5, 0.5, 0, 0))
    bleu3 = bleu_score(predicted, target, weights=(1/3, 1/3, 1/3, 0))
    bleu4 = bleu_score(predicted, target)
    
    return bleu1, bleu2, bleu3, bleu4

In [26]:
def evaluate_model(enc_obj, dec_obj, source_test, target_test, target_vocab_dict):
    
    # returns the average bleu score for the model with the given test data
    
    BLEU_1 = 0
    BLEU_2 = 0
    BLEU_3 = 0
    BLEU_4 = 0
    for i in range(len(source_test)):
        source_sentence = torch.cuda.LongTensor(source_test[i])
        target = [target_vocab_dict[x] for x in target_test[i][:-1]]
        bleu1, bleu2, bleu3, bleu4 = eval_bleu(enc_obj, dec_obj, source_sentence, target, target_vocab_dict)
        BLEU_1 += bleu1
        BLEU_2 += bleu2
        BLEU_3 += bleu3
        BLEU_4 += bleu4
    
    return BLEU_1, BLEU_2, BLEU_3, BLEU_4

In [27]:
evaluate_model(encoder, decoder, X_test, Y_test, TAM.index_word)

Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['நீங்கள்', 'எங்களுக்கு', 'கோப்ரா', 'என்ற', 'சொல்கிறேன்', 'அல்லது', 'die', 'நீ', 'நம்', 'மாஸ்டர்', 'கொல்ல', 'அதே', 'வாள்']]]
Predicted: [['ஒரு', 'ஒரு']]
Target: [[['அவர்கள்', 'அதே', 'படுக்கை', 'அதே', 'எழுந்து', 'அதே', 'மன', 'ஒரு', 'எரிவாயு', 'நிலையம்']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['என்ன', 'அன்று', 'இரவு', 'என்ன', 'நடந்தது', 'மறக்க', 'ஒரே', 'ஒரு', 'கணம்', 'இருந்தது']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['நீல்ஸ்', 'வேண்டும்', 'அடுத்த', 'எப்படியோ', 'நான்', 'பழைய', 'கொட்டகையின்', 'வேண்டும்', 'எல்', 'செய்']]]
Predicted: [['நீங்கள்', 'ஒரு']]
Target: [[['செம்மை', 'எந்த', 'அதிர்ஷ்டம்', 'இருந்தது', 'அதனால்', 'அவர்கள்', 'நமக்கு', 'கேட்டார்', 'இது', 'ஒரு', 'கிராக்', 'எடுத்து']]]
Predicted: [['நான்', 'நீங்கள்', 'நீங்கள்']]
Target: [[['நீங்கள்', 'அவரை', 'ஒரு', 'பெயர்', 'கொடுத்து', 'இருந்தால்', 'அவர்', 'ஒருவேளை', 'நல்ல', 'என்றார்', 'நீங்கள்', 'செல்லலாம்']]]
Predicted: [['ஒரு', 'ஒரு', 'ஒரு']]
Target: [[['போன்ற

Predicted: [['ஒரு', 'ஒரு', 'ஒரு']]
Target: [[['பிரமை', 'ஒரு', 'விஷயம்', 'ஆனால்', 'நீங்கள்', 'குழந்தைகள்', '1', 'நாள்', 'நீடிக்கும்', 'வாட்டு', 'உள்ள', 'முடியாது']]]
Predicted: [['நான்', 'நான்', 'நான்']]
Target: [[['நீ', 'தொலைக்காட்சி', 'பார்த்துள்ளனர்', 'இது', 'காட்சி', 'அமர்ந்து', 'போது', 'நான்', 'மூலம்', 'தொலைக்காட்சி', 'கண்ணாடி', 'பார்த்து']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['ஆனால்', 'போதுமான', 'நீங்கள்', 'ஓட்ட', 'நீங்கள்', 'நீங்கள்', 'ஏமாற்றி', 'வருகின்றன']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['என்', 'எதிரி', 'ஆனால்', 'என்', 'எனக்கு', 'தேர்வு', 'செய்யப்பட்டார்', 'வன்', 'மாஸ்டர்', 'கொலை', 'என்னை', 'யார்']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['அது', 'சரி', 'இப்போது', 'அந்த', 'விடு', 'என்னுடைய', 'கடின', 'உழைப்பு', 'உங்களை', 'வாங்க']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['um', 'சரி', 'ஒருவேளை', 'நீங்கள்', 'எனக்கு', 'ஒரு', 'சில', 'கேள்விகளுக்கு', 'பதில்', 'சொல்ல', 'முடியும்']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Targ

Predicted: [['ஒரு', 'ஒரு']]
Target: [[['இது', 'இரத்த', 'என்', 'தலை', 'வழியாக', 'அவசரமாக', 'அல்லது', 'குலுக்கல்', 'எந்த', 'செய்யும்', 'இல்லை']]]
Predicted: [['ஒரு', 'ஒரு']]
Target: [[['மற்றும்', 'கொண்டு', 'இராணுவ', 'படைகள்', 'இந்தியா', 'சீனா', 'மற்றும்', 'ரஷ்யா', 'அவசரகால', 'நிலையை', 'செய்தார்']]]
Predicted: [['நீங்கள்', 'ஒரு']]
Target: [[['அப்போது', 'நீங்கள்', 'இருந்த', 'போது', 'ஒரு', 'சிறைப்பிடிக்கப்பட்ட', 'banir', 'வலது', 'ஒரு', 'பெயர்', 'வேண்டும்']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['அவளை', 'மாதத்திற்கு', 'மருத்துவ', 'அவர்கள்', 'fucking', 'நிறுத்த', 'செலுத்தும்', 'அதை', 'நீங்கள்', 'மிகவும்', 'உடம்பு', 'பெற', 'இது', 'மிகவும்', 'செலவு', 'என்றால்', 'என்கிறார்']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['அதை', 'நீங்கள்', 'ஓய்வு', 'எடுக்கும்', 'கூட', 'உங்கள்', 'வாழ்க்கையில்', 'நீங்கள்', 'கடமைப்பட்டிருக்கிறேன்', 'என்ன', 'காரணம்', 'என்று', 'கண்டுபிடிக்க']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['சரி', 'இங்க', 'இருக்கிற', 'ஒவ்வொரு', 'உடைந்த', '

Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['அது', 'நீண்ட', 'திறந்த', 'இருக்க', 'பின்னர்', 'அதை', 'மற்றும்', 'banir', 'அதனை', 'விட்டிருக்கும்', 'என்று']]]
Predicted: [['நான்', 'நான்', 'நான்']]
Target: [[['ஆனால்', 'நான்', 'உன்னை', 'காதலித்தேன்', 'நான்', 'நினைவில்', 'முடியும்', 'வரை', 'என', 'அமெரிக்கா']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['இது', 'அல்பேனியர்கள்', 'ஒன்றாகும்', 'நபர்', 'நான்', 'அதை', 'சொல்ல', 'அவரை', 'வேண்டும்', 'நான்', 'சொல்ல', 'அவருக்கு', 'தேவையான', 'ஒன்று', 'என்று']]]
Predicted: [['ஒரு', 'ஒரு']]
Target: [[['நிலவொளி', 'மற்றும்', 'முத்தம்', 'சோக', 'புல்லாங்குழல்', 'என்ற', 'தாளத்துக்கு', 'நட்சத்திரங்கள்', 'கீழ்', 'நீங்கள்', 'ஈ', 'ஆகிய']]]
Predicted: [['நான்', 'நீங்கள்', 'நீங்கள்']]
Target: [[['அதை', 'நீங்கள்', 'சிறை', 'அல்ல', 'நான்', 'போலீஸ்', 'அழைத்து', 'நான்', 'பார்த்தேன்', 'என்ன', 'இன்னும்', 'சொல்ல', 'இப்போது', 'உணர', 'வேண்டும்']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['என்ன', 'முக்கியம்', 'நான்', 'வெற்றி', 'என்று', 'நான்', '

Predicted: [['நான்', 'நீங்கள்', 'நீங்கள்']]
Target: [[['உண்மை', 'அவர்', 'தான்', 'எப்போதும்', 'அங்கு', 'இருந்தது', 'நான்', 'உண்மையில்', 'அவர்', 'என்ன', 'என்று', 'எனக்கு', 'தெரியாது', 'ஆகும்']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'நீங்கள்']]
Target: [[['நீங்கள்', 'இழந்த', 'போது', 'அல்லது', 'மறைக்க', 'நீங்கள்', 'புகழ்', 'கொண்டு', 'கட்டி', 'தழுவு', 'முடியாது', 'அது', 'தனியாக', 'இருக்கிறது', 'மேல்']]]
Predicted: [['ஒரு', 'ஒரு']]
Target: [[['அந்த', 'அசுத்தத்தை', 'greely', 'தெரிந்தது', 'அவர்', 'எங்களை', 'அச்சுறுத்த', 'பொருட்டு', 'காப்பீடு', 'சில', 'வகையான', 'தேவை']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['அவர்களுக்கு', 'வேலை', 'அவரை', 'put', 'முடியும்', 'மற்றும்', 'பின்னர்', 'அவர்கள்', 'பணியில்', 'மற்ற', 'மக்கள்', 'put', 'முடியும்']]]
Predicted: [['நான்', 'நீங்கள்', 'நீங்கள்']]
Target: [[['நான்', 'இப்போது', 'ஒரு', 'சொந்த', 'வணிகம்', 'வைத்திருக்கிறேன்', 'என்பதை', 'அந்த', 'பணம்', 'நீ', 'இருந்து', 'பெற்றது', 'என்னுடையது']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['நான்

...

Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['நான்', 'வேண்டும்', 'குழுவில்', 'சம', 'அந்தஸ்து', 'வேண்டும்', 'மற்றபடி', 'சைமன்', 'என்னை', 'மதிக்க', 'மாட்டான்']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['அவர்', 'என்', 'குழந்தையின்', 'வாயில்', 'உணவு', 'வெளியே', 'எடுத்து', 'என்', 'குழந்தை', 'இருக்கிறது']]]
Predicted: [['நான்', 'நான்', 'நான்', 'நான்']]
Target: [[['ஆமாம்', 'நான்', 'அவன்', 'இல்லை', 'என்று', 'நினைக்கிறேன்', 'இறுதியாக', 'நடக்கும்', 'பார்க்க', 'இங்கே']]]
Predicted: [['நீங்கள்', 'ஒரு', 'ஒரு']]
Target: [[['நாங்கள்', 'முதலில்', 'நேரடி', 'வெளியே', 'உள்ளன', 'இரண்டு', 'வைத்திருக்கும்', 'எங்கே', 'பத்து', 'பேர்', 'ஒரு', 'கையில்', 'துப்பாக்கி', 'பணயக்கைதிகள்']]]
Predicted: [['நீங்கள்', 'ஒரு', 'ஒரு']]
Target: [[['இது', 'ஒரு', 'தந்திரமான', 'செயல்', 'நாங்கள்', 'கண்காணிக்க', 'வேண்டும்', 'உங்கள்', 'நாம்', 'சரியான', 'கலவை', 'கண்டுபிடிக்க', 'வரை']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['நான்', 'இங்கே', 'இருக்கிறேன்', 'மட்டுமே', 'காரணம்', 'நாங்கள்', 'மண்ணில்', 'தான்', 'காரணம்']

Predicted: [['நான்', 'நான்', 'நான்', 'நான்']]
Target: [[['நாம்', 'அதை', 'வேறு', 'யாராவது', 'கொண்டு', 'முன்பு', 'நான்', 'ஒரு', 'நல்ல', 'இடத்தில்', 'இருக்க', 'வேண்டும்', 'என்று', 'நினைக்கிறேன்']]]
Predicted: [['நான்', 'நீங்கள்', 'நீங்கள்']]
Target: [[['நான்', 'இன்னும்', 'போதிலும்', 'இரவு', 'உணவு', 'மற்றும்', 'விஷயங்களை', 'செலுத்த', 'போதுமான', 'பணம்', 'இல்லை']]]
Predicted: [['நான்', 'நான்', 'நான்']]
Target: [[['நான்', 'நதிங்', 'இது', 'பற்றி', 'இல்லை', 'போது', 'ஒரு', 'முறை', 'இருந்தது', 'எனக்கு']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['சரி', 'ஃபிலிப்', 'நாம்', 'இந்த', 'ஒப்பந்தத்தை', 'முன்', 'கேட்கும்', 'விலையில்', 'இப்போது', 'நாம்', 'இப்போது', 'அந்த', 'குளியலை', 'சோதிக்க', 'வேண்டும்']]]
Predicted: [['நான்', 'நீங்கள்', 'நீங்கள்']]
Target: [[['ஒரு', 'உண்மையான', 'மோசமான', 'இரவு', 'வேண்டும்', 'என்று', 'நான்', '9', '00', 'மணிக்கு', 'அந்த', 'கதவு', 'வழியாக', 'நடந்து', 'எவர்', 'கூறினார்']]]
Predicted: [['நீங்கள்', 'ஒரு', 'ஒரு']]
Target: [[['பீர்', 'வெளில', 'இருக்கு', 'பென்',

Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['நாம்', 'இதை', 'சரி', 'பெற', 'முடியும்', 'வரை', 'பேப்', 'நாம்', 'பயன்படுத்தும்', 'அட்டைகள்', 'துடைக்க', 'வேண்டும்']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['நாம்', 'வெற்றி', 'போது', 'நாம்', 'வெற்றி', 'கிடைத்தது', 'எப்படி', 'நாம்', 'ஆரம்பத்தில்', 'இருந்து', 'வலது', 'அமைக்கப்பட்டன']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['சரி', 'நல்லது', 'கூப்பிட்டு', 'சொல்றேன்', 'ஆனா', 'நீங்க', 'பணம்', 'அந்த', 'மாற்ற']]]
Predicted: [['நான்', 'நான்', 'நான்']]
Target: [[['நான்', 'கண்ணாடியை', 'சரி', 'நான்', 'ஆய்வு', 'செய்தேன்', 'அவர்கள்', 'முழு', 'இருக்க', 'விரும்புகிறேன்', 'போன்ற', 'அவர்கள்', 'பார்த்து', 'சரியா']]]
Predicted: [['நீங்கள்', 'நீங்கள்', 'ஒரு']]
Target: [[['எனக்குத்', 'தெரியும்', 'இல்ல', 'வேலை', 'போறது']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['எனவே', 'நீங்கள்', 'பார்க்க', 'முடியும்', 'என்றால்', 'வேறு', 'வழி', 'இந்த', 'தொல்லை', 'சமாளிக்க', 'நான்', 'கேட்க', 'விரும்புகிறேன்']]]
Predicted: [['ஒரு', 'ஒரு']]
Target

Predicted: [['நான்', 'நான்', 'நான்']]
Target: [[['பாதுகாப்பாக', 'கட்டிடம்', 'வெளியே', 'அழைத்து', 'வருகின்றனர்', 'நான்', 'திரு', 'போபோ', 'மற்றும்', 'அவரது', 'மகள்', 'என்பதை', 'உறுதி', 'செய்ய', 'வேண்டும்']]]
Predicted: [['நான்', 'நான்', 'நான்', 'நான்']]
Target: [[['நீங்கள்', 'இந்த', 'நேரத்தில்', 'இப்போது', 'இது', 'சரியான', 'செய்ய', 'ஏன்', 'என்று', 'எனக்கு', 'புரியவில்லை']]]
Predicted: [['ஒரு', 'ஒரு']]
Target: [[['நீங்கள்', 'அவரை', 'உங்கள்', 'முழு', 'உடைந்த', 'சரங்களை', 'கட்டி', 'களிமண்', 'சீர்', 'செய்யப்படும்']]]
Predicted: [['ஒரு', 'ஒரு']]
Target: [[['நாள்', 'மூன்று', 'மூலம்', 'அவர்', 'ஏற்கனவே', 'எப்படியாவது', 'வரை', 'உட்கார்ந்து', 'மற்றும்', 'சுவரில்']]]
Predicted: [['நான்', 'நீங்கள்', 'நீங்கள்']]
Target: [[['எப்படி', 'நீங்கள்', 'எப்படி', 'பாலியல்', 'இணக்க', 'தெரியும்', 'நீங்கள்', 'அதை', 'ஒப்பிட்டு', 'இல்லை', 'போது']]]
Predicted: [['நீங்கள்', 'நீங்கள்']]
Target: [[['அதை', 'நீங்கள்', 'தெளிவான', 'குழந்தைகள்', 'முக்கிய', 'இருந்தன', 'நீங்கள்', 'நோய்', 'எதிர்ப்பு', 'என்பதால்']]]
Predic

(0.0, 0.0, 0.0, 0.0)